In [4]:
pip install apache-beam

Note: you may need to restart the kernel to use updated packages.


In [6]:
import apache_beam as beam
import pandas as pd
import numpy as np

ds1 = pd.DataFrame({
    'invoice_id' : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
    'legal_entity' : ['L1','L2','L3','L1','L2','L3','L1','L2','L3','L1','L2','L3','L1','L2','L3','L1','L2','L3'],
    'counter_party' : ['C1','C2','C3','C4','C5','C6','C1','C2','C3','C4','C5','C6','C1','C3','C3','C3','C5','C6'],
    'rating' : [1,2,4,6,4,6,2,3,3,5,6,4,3,2,4,6,3,5],
    'status' : ['ARAP','ARAP','ACCR','ARAP','ACCR','ACCR','ARAP','ACCR','ACCR','ACCR','ARAP','ARAP','ARAP','ACCR','ACCR','ARAP','ACCR','ARAP'],
    'value' : [10,20,30,40,50,60,10,40,80,100,1000,80,20,52,35,5,65,65],
})

ds2 = pd.DataFrame({
    'counter_party' : ['C1','C2','C3','C4','C5','C6'],
    'tier' : [1,2,3,4,5,6],
})


In [8]:
ds = pd.merge(ds1, ds2, on='counter_party', how='left')
ds

,invoice_id,legal_entity,counter_party,rating,status,value,tier
0,1,L1,C1,1,ARAP,10,1
1,2,L2,C2,2,ARAP,20,2
2,3,L3,C3,4,ACCR,30,3
3,4,L1,C4,6,ARAP,40,4
4,5,L2,C5,4,ACCR,50,5
5,6,L3,C6,6,ACCR,60,6
6,7,L1,C1,2,ARAP,10,1
7,8,L2,C2,3,ACCR,40,2
8,9,L3,C3,3,ACCR,80,3
9,10,L1,C4,5,ACCR,100,4


In [9]:
grouped_ds = ds.groupby(['legal_entity', 'counter_party', 'tier']).agg({
    'rating': 'max',
    'value': ['sum'],
    'status': ['sum']
})

grouped_ds

rating value        status
                                   max   sum           sum
legal_entity counter_party tier                           
L1           C1            1         3    40  ARAPARAPARAP
             C3            3         6     5          ARAP
             C4            4         6   140      ARAPACCR
L2           C2            2         3    60      ARAPACCR
             C3            3         2    52          ACCR
             C5            5         6  1115  ACCRARAPACCR
L3           C3            3         4   145  ACCRACCRACCR
             C6            6         6   205  ACCRARAPARAP

In [20]:
p = beam.Pipeline()

In [18]:
ds_pcoll = p | 'Create' >> beam.Create(grouped_ds.to_dict('index').values())

In [19]:
formatted_pcoll = ds_pcoll | 'Format' >> beam.Map(lambda x: (x[0][0], x[0][1], x[0][2], x[1]['rating']['max'], x[1]['value']['sum']['ARAP'], x[1]['value']['sum']['ACCR']))

In [21]:
_ = formatted_pcoll | 'Write' >> beam.io.WriteToText('output.csv')

In [22]:
result = p.run()
result.wait_until_finish()

'DONE'